In [1]:
# import sys
# sys.path.insert(0, '../../src')

import quik_ai as qa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from importlib.metadata import version 
print('Quik-AI Version: %s' % version('quik_ai'))

Quik-AI Version: 1.2.0


In [2]:
# build univariate time-series data by combining random frequencies
np.random.seed(123)

# creating the dataframes
domain = np.linspace(0, 80 * np.pi, 1000)
df1 = pd.DataFrame({'value': np.sin(domain), 'func': 'sin'})
df2 = pd.DataFrame({'value': np.cos(domain), 'func': 'cos'})
df3 = pd.DataFrame({'value': np.sin(domain)*np.cos(domain), 'func': 'sincos'})

# resetting the index to create an auxiliary column for each dataframe
df1 = df1.reset_index()
df2 = df2.reset_index()
df3 = df3.reset_index()

# concatenating the dataframes
data = pd.concat([df1, df2, df3])

# Sorting by the auxiliary index and then removing the auxiliary column
data = data.sort_values(by='index').reset_index(drop=True)
data = data.drop(columns='index')

# plot the two time-series
cnt = 100
plt.plot(data.index[-cnt:], data.tail(cnt).value, label='series');
plt.xlabel('time');
plt.legend();

In [3]:
# split the data
training_data, validation_data, test_data = qa.train_val_test_split(data)

# create the driver, this will generate the tensorflow dataset and feed data
driver = qa.Driver(training_data, validation_data, test_data, time_group_column='func')

# what does our response look like?
head = qa.heads.Regression()

# all the predictors for the model, in this case we only have one
# we will mask out the value in the current row while looking at
# the prior data
predictors = [
    qa.predictors.TimeMasked('value'),
]

# what is our model architecture? This will store
# our model, driver, and any other information we use in training
# testing, saving, and loading needed for this problem
model = qa.models.RNN('value', head, predictors, driver, time_window=32)

# train the model, we can provide a hyperparameter tuner and other
# training parameters, by default we will use a custom BOHB tuner
model.train(qa.tuners.BOHB, tuner_params={'max_epochs' : 10})

Trial 30 Complete [00h 00m 21s]
val_mean_squared_error: 0.07768413424491882

Best val_mean_squared_error So Far: 0.00688123470172286
Total elapsed time: 00h 04m 40s
INFO:tensorflow:Oracle triggered exit
Round 1 best score: 0.0045


In [4]:
# evaluate the model on test data
model.evaluate()

1/1 [==============================] - 0s 241ms/step - loss: 0.0045 - mean_squared_error: 0.0045


{'loss': 0.00452654343098402, 'mean_squared_error': 0.00452654343098402}

In [5]:
# save the model to disk and reload the model
# then test on the test data once again
model.save()
model = qa.HyperModel.load()

# evaluate again, this should be the same score as before
model.evaluate()

INFO:tensorflow:Assets written to: ./model\tf_model\assets


1/1 [==============================] - 1s 608ms/step - loss: 0.0045 - mean_squared_error: 0.0045


{'loss': 0.00452654343098402, 'mean_squared_error': 0.00452654343098402}

In [6]:
# get predictions on the test
predictions = model.predict(test_data, verbose=0)

# get actuals
actuals = test_data.value

# plot the predictions vs forecast
plt.plot(test_data.index[-cnt:], actuals.tail(cnt), label='actual');
plt.plot(test_data.index[-cnt:], predictions[-cnt:], 'r+', label='forecast');
plt.xlabel('time')
plt.legend();

In [7]:
training_data.head(20)

,value,func
0,0.000000,sin
1,1.000000,cos
2,0.000000,sincos
3,0.248934,sin
4,0.968521,cos
5,0.241097,sincos
6,0.482195,sin
7,0.876064,cos
8,0.422433,sincos
9,0.685097,sin


In [8]:
driver = qa.Driver(training_data, validation_data, test_data, time_group_column='func', shuffle=False, batch_size=20)

tdf = driver.get_training_tensorflow_dataset(['value'], None, time_window=5, hp=None)

for x in tdf:
    print(x)
    break

{'value': <tf.Tensor: shape=(20, 5, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [1.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.24893355]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [1.        ],
        [0.9685206 ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.24109727]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.24893355],
        [0.48219454]],

       [[0.        ],
        [0.        ],
        [1.        ],
        [0.9685206 ],
        [0.8760642 ]],

       [[0.        ],
        [0.        ]